In [ ]:
%load_ext autoreload
%autoreload 2

# setup

> Setup directories to store data. Download reference genomes, chain files.

In [ ]:
#| default_exp setup

In [ ]:
#| hide
from nbdev.showdoc import *

/home/magyary/anaconda3/envs/kerepesi_2025/lib/python3.12/site-packages/nbdev/doclinks.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources,importlib


# Download reference genomes

In [ ]:
#| export

from pathlib import Path
import os, json
import requests
from tqdm.auto import tqdm

In [ ]:
#| export 
CONFIG_FILE = Path.home() / ".bs-cpg-config.json"
SUBDIR_NAME = ".bs-cpg"

def get_base_data_path():
    if CONFIG_FILE.exists():
        base = Path(json.loads(CONFIG_FILE.read_text())["base_path"]).expanduser()
    else:
        user_input = input("Where should .bs-cpg be stored (e.g. ~ or /mnt/data)? ")
        base = Path(user_input).expanduser().resolve()
        CONFIG_FILE.write_text(json.dumps({"base_path": str(base)}))
    
    final_path = base / SUBDIR_NAME
    final_path.mkdir(parents=True, exist_ok=True)
    return final_path
get_base_data_path()

Path('/mnt/idms/home/magyary/.bs-cpg')

In [ ]:
#| export 
def download_file(url: str, filename: str, sub_dir: Path = None):
    """
    A general utility to download a file with a progress bar.

    Args:
        url (str): The URL of the file to download.
        filename (str): The name for the saved file.
        sub_dir (Path): The subdirectory where the file will be saved under get_base_data_path().
            If empty, it will be saved to get_base_data_path().
        
    Returns:
        Path: The full path to the downloaded file, or None on error.
    """
    # Ensure the destination directory exists
    base_dir = get_base_data_path()
    dest_dir = base_dir / sub_dir if sub_dir else base_dir
    dest_dir.mkdir(parents=True, exist_ok=True)
    dest_path = dest_dir / filename

    if dest_path.exists():
        print(f"✅ File '{dest_path}' already exists. Skipping.")
        return dest_path

    print(f"⬇️  Downloading '{filename}' from {url}...")
    try:
        response = requests.get(url, stream=True, timeout=15)
        response.raise_for_status()  # Raise an exception for bad status codes

        total_size = int(response.headers.get('content-length', 0))

        with open(dest_path, 'wb') as f, tqdm(
            desc=filename,
            total=total_size,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for chunk in response.iter_content(chunk_size=8192):
                size = f.write(chunk)
                bar.update(size)

        print(f"✅ Successfully downloaded to '{dest_path}'")
        return dest_path

    except requests.exceptions.RequestException as e:
        print(f"❌ Error downloading file: {e}")
        if dest_path.exists():
            dest_path.unlink()  # Clean up partial file
        return None

In [ ]:
#| export 
def download_ref_genome(name: str):
    """
    Download a reference genome from UCSC goldenPath bigZips.

    Args:
        name (str): The reference genome name (e.g., 'hg19', 'hg38', 'mm10').
    
    Returns:
        Path: The path to the downloaded file, or None if an error occurred.
    """
    file_name = f"{name}.fa.gz"
    url = f"https://hgdownload.soe.ucsc.edu/goldenPath/{name}/bigZips/{file_name}"
    
    return download_file(url=url, filename=file_name)
download_ref_genome('hg38')

✅ File '/mnt/idms/home/magyary/.bs-cpg/hg38.fa.gz' already exists. Skipping.


Path('/mnt/idms/home/magyary/.bs-cpg/hg38.fa.gz')

In [ ]:
"hg38".capitalize()

'Hg38'

In [ ]:
#| export 
def download_liftover_chain(genome_from: str, genome_to: str):
    """
    Download liftover chain file between genome versions from UCSC goldenPath liftOver.

    Args:
        genome_from (str): The original reference genome name (e.g., 'hg19', 'hg38', 'mm10').
        genome_to (str): The new reference genome name (e.g., 'hg19', 'hg38', 'mm10').
    
    Returns:
        Path: The path to the downloaded file, or None if an error occurred.
    """
    genome_to_camel=genome_to.capitalize()
    file_name = f"{genome_from}To{genome_to_camel}.over.chain.gz"
    url = f"https://hgdownload.soe.ucsc.edu/goldenPath/{genome_from}/liftOver/{file_name}"

    return download_file(url=url, filename=file_name)
download_liftover_chain("hg19", "hg38")

⬇️  Downloading 'hg19ToHg38.over.chain.gz' from https://hgdownload.soe.ucsc.edu/goldenPath/hg19/liftOver/hg19ToHg38.over.chain.gz...


hg19ToHg38.over.chain.gz:   0%|          | 0.00/222k [00:00<?, ?iB/s]

✅ Successfully downloaded to '/mnt/idms/home/magyary/.bs-cpg/hg19ToHg38.over.chain.gz'


Path('/mnt/idms/home/magyary/.bs-cpg/hg19ToHg38.over.chain.gz')

In [ ]:
#| export

def func(): pass;

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()